In [2]:
using Distributed

### Preparo los workers

In [3]:
workervec = [("ubuntu@18.221.214.62:22", 2)]
addprocs(workervec; sshflags=`-i ./JuliaServer.pem`, tunnel=true, exename=`/home/ubuntu/julia-1.2.0/bin/julia`, dir="/home/ubuntu/tdl-2c2019")


      From worker 2:	ERROR: could not load library "/home/ubuntu/julia-1.2.0/lib/julia/sys.so"
      From worker 2:	/home/ubuntu/julia-1.2.0/lib/julia/sys.so: failed to map segment from shared object


CompositeException: On worker 2:
Unable to read host:port string from worker. Launch command exited with error?
read_worker_host_port at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/cluster.jl:320
launch_additional at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/cluster.jl:679
#112 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:292
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:79
macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:292 [inlined]
#111 at ./task.jl:268
#remotecall_fetch#149(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::typeof(remotecall_fetch), ::Function, ::Distributed.Worker, ::Int64, ::Vararg{Any,N} where N) at /home/buildbot/build-worker/worker/juliap

In [5]:
addprocs(2)

2-element Array{Int64,1}:
 3
 4

In [6]:
@everywhere using Distributed
@everywhere using PyCall
println("Qty workers: ", workers())

Qty workers: [2, 3, 4]


In [7]:
@everywhere function listenChannel(channel)
    println("Listen Started")
    id = 1
    while(id != -1)
        id = take!(channel)
        message = string("I'm process: ", string(myid()), " and i read element: ", id, " from the channel")
        println(message)
    end
    println(string("i'm process: ", string(myid()), " and i'm finished listening"))
end

### Crea un canal remoto, para el pid 1(el main process) que alojara un canal vacio de 1000 Strings


In [8]:
channel = RemoteChannel(() -> Channel{String}(1000), 1)
[@spawnat i listenChannel(channel) for i in workers()]

3-element Array{Future,1}:
 Future(2, 1, 31, nothing)
 Future(3, 1, 32, nothing)
 Future(4, 1, 33, nothing)

      From worker 4:	Listen Started
      From worker 3:	Listen Started
      From worker 2:	Listen Started


### pero ahora populo el canal para que los procesos extraigan del canal

In [9]:
function fillChannel(channel, size)
    for i=1:size
        put!(channel, string("Elemento: ", i))
    end

    for i=1:length(workers())
        put!(channel, string(-1))
    end
end

fillChannel (generic function with 1 method)

In [10]:
println("filling the channel")
fillChannel(channel, 16)


filling the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 2 from the channel
      From worker 3:	I'm process: 3 and i read element: Elemento: 1 from the channel
      From worker 3:	I'm process: 3 and i read element: Elemento: 4 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 5 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 6 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 7 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 8 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 9 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 10 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 11 from the channel
      From worker 4:	I'm process: 4 and i read element: Elemento: 12 from the channel
      From worker 3:	I'm process: 3 and i 